In [2]:
from openmoltools.utils import smiles_to_mdtraj_ffxml
from openmoltools.amber import run_antechamber, find_gaff_dat
from openmoltools.amber_parser import AmberParser

In [3]:
import tempfile
from rdkit import Chem
from rdkit.Chem import AllChem

def antechamber_from_smiles(
    smiles_string, molecule_name, frcmod_filename,
    gaff_mol2_filename=None, net_charge=None, charge_method="bcc", 
    resname=False, log_debug_output=False, gaff_version='gaff'):

    input_format = 'mdl'

    m = Chem.MolFromSmiles(smiles_string)
    m = Chem.AddHs(m)
    AllChem.EmbedMolecule(m)
    AllChem.UFFOptimizeMolecule(m)

    mdl_filename = tempfile.mktemp(suffix='.%s' % input_format)
    Chem.MolToMolFile(m, mdl_filename)

    return run_antechamber(
        molecule_name, mdl_filename, charge_method, 
        net_charge, gaff_mol2_filename, frcmod_filename, 
        input_format, resname, log_debug_output, gaff_version)

def create_ffxml_file(
    gaff_mol2_filename, frcmod_filename, ffxml_filename,
    override_mol2_residue_name=None):
    
    parser = AmberParser(override_mol2_residue_name=override_mol2_residue_name)
    parser.parse_filenames([find_gaff_dat(), gaff_mol2_filename, frcmod_filename])

    ffxml_stream = parser.generate_xml()

    if ffxml_filename is not None:
        outfile = open(ffxml_filename, 'w')
        outfile.write(ffxml_stream.read())
        outfile.close()
        ffxml_stream.seek(0)

    return ffxml_stream

In [6]:
(FAD_mol2_file, FAD_frcmod_file) = antechamber_from_smiles(
    smiles_string='CC1=CC2=C(C=C1C)N(C3=NC(=O)NC(=O)C3=N2)C[C@@H]([C@@H]([C@@H](COP(=O)(O)OP(=O)(O)OC[C@@H]4[C@H]([C@H]([C@@H](O4)N5C=NC6=C(N=CN=C65)N)O)O)O)O)O',
    molecule_name='FAD', 
    frcmod_filename='./FAD.frcmod',
    gaff_mol2_filename='./FAD.gaff.mol2')

FAD_ffxml_file = create_ffxml_file(
    FAD_mol2_file, FAD_frcmod_file, 
    ffxml_filename='./FAD.ffxml',
    override_mol2_residue_name='FAD')

In [7]:
(CBGA_mol2_file, CBGA_frcmod_file) = antechamber_from_smiles(
    smiles_string='CCCCCC1=CC(=C(C(=C1C(=O)O)O)C/C=C(\C)/CCC=C(C)C)O',
    molecule_name='CBGA', 
    frcmod_filename='./CBGA.frcmod',
    gaff_mol2_filename='./CBGA.gaff.mol2')

CBGA_ffxml_file = create_ffxml_file(
    CBGA_mol2_file, CBGA_frcmod_file, 
    ffxml_filename='./CBGA.ffxml',
    override_mol2_residue_name='CBGA')

In [ ]:
%%bash
cp2k.sopt em.inp > em.out

In [ ]:
%%bash
parmed complex.prmtop
changeLJSingleType :WAT@H1 0.3019 0.047
changeLJSingleType :*@HO 0.3019 0.047
outparm complex_LJ_mod.prmtop
quit

In [ ]:
%%bash
cp2k.sopt md.inp